In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import time
import json


In [13]:
#Funkcja do otwarcia url i zamiana na jason
def get_json(url):

    headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"}

    resp = requests.get(url,headers=headers)

    soup = BeautifulSoup(resp.text,'html.parser')
    data_tag = soup.find('script',{'id':'__NEXT_DATA__'}).text
    
    data = json.loads(data_tag)

    return data 

#Funkcja do ściągnięcia linków do stron 
def pages_to_scrape(pages):
    Lista_url= []
    for page in range(1,pages+1):

        search_url = f'https://www.otodom.pl/pl/oferty/sprzedaz/mieszkanie/warszawa?page={page}&by=LATEST&direction=ASC&limit=36'
        search_data = get_json(search_url)

        for prop in search_data['props']['pageProps']['data']['searchAds']['items']:
            print(prop['title']) #prop has the property summary - not everything
            
            prop_url = 'https://www.otodom.pl/pl/oferta/'+prop['slug']
            print(prop_url)
            Lista_url.append(prop_url)
    print('skończone')
    return Lista_url



In [32]:
#Wywołanie funkcji
Otodom_links = pages_to_scrape(10)


Mieszkanie, 200 m², Warszawa
https://www.otodom.pl/pl/oferta/mieszkanie-200-m-warszawa-ID2f1sl
Mieszkanie, 113,80 m², Warszawa
https://www.otodom.pl/pl/oferta/mieszkanie-113-80-m-warszawa-ID2OsgW
GOTOWE 75m2 + poddasze 90m2 ul. Tawułkowa 7ZC
https://www.otodom.pl/pl/oferta/gotowe-75m2-poddasze-90m2-ul-tawulkowa-7zc-ID2VUZD
Cztery pokoje na Mokotowie przy parku.
https://www.otodom.pl/pl/oferta/cztery-pokoje-na-mokotowie-przy-parku-ID3ciQ6
Pola Mokotowskie! Apartament z ogrodem na dachu !
https://www.otodom.pl/pl/oferta/pola-mokotowskie-apartament-z-ogrodem-na-dachu-ID3fICS
Gotowy Apartament - 2 pokoje 38m2 z meblami
https://www.otodom.pl/pl/oferta/gotowy-apartament-2-pokoje-38m2-z-meblami-ID3hWDC
Jedyne takie dwa ogródki 30/15 m2 , tuż przy park
https://www.otodom.pl/pl/oferta/jedyne-takie-dwa-ogrodki-30-15-m2-tuz-przy-park-ID3yjyD
Apartament z Recepcją blisko Centrum ! WYKOŃCZONY
https://www.otodom.pl/pl/oferta/apartament-z-recepcja-blisko-centrum-wykonczony-ID3C7RK
Mieszkanie, 135 m²,

In [33]:
len(Otodom_links)

360

In [34]:
#Ustawianie kluczy i tworzenie pustego słownika
klucze_target = ['Area', 'Build_year', 'Building_floors_num', 'Building_type', 'Building_ownership', 'Construction_status', 'City', 'City_id', 'Country', 'Extras_types', 'Floor_no', 'MarketType', 'OfferType', 'Price', 'Price_per_m', 'ProperType', 'Province', 'Rent', 'Rooms_num', 'Subregion', 'Windows_type', 'env', 'Equipment_types', 'Heating', 'Media_types', 'Security_types', 'Building_material', 'user_type']
klucze_ad = ['id', 'market', 'slug', 'advertType', 'createdAt', 'modifiedAt', 'description', 'exlusiveOffer']
klucze_owner = ['id', 'name', 'phones']
klucze_agency = ['id', 'name']
klucze_location_coordinates = ['latitude', 'longitude']
klucze_district = ['code', 'id', 'name']
wszystkie_mieszkanie = []

#Tworzenie petli, zeby przechodzilo, przez wszystki linki
for link in Otodom_links:
    search_data = get_json(link)
    mieszkanie = {}
    # Pętla dla kluczy w sekcji 'target'
    for klucz in klucze_target:
        mieszkanie[klucz] = search_data['props']['pageProps']['ad']['target'].get(klucz)

    # Pętla dla kluczy w sekcji 'ad'
    for klucz in klucze_ad:
        mieszkanie[klucz] = search_data['props']['pageProps']['ad'].get(klucz)

    # Pętla dla kluczy w sekcji 'owner'
    for klucz in klucze_owner:
        mieszkanie['owner_' + klucz] = search_data['props']['pageProps']['ad']['owner'].get(klucz)

    #Pętla dla kluczy w sekcji 'agency'
    if search_data['props']['pageProps']['ad']['agency'] is not None:
        for klucz in klucze_agency:
            mieszkanie['agency_' + klucz] = search_data['props']['pageProps']['ad']['agency'].get(klucz)
    

    # Pętla dla kluczy w sekcji 'coordinates'
    for klucz in klucze_location_coordinates:
        mieszkanie[klucz] = search_data['props']['pageProps']['ad']['location']['coordinates'].get(klucz)

   # #Pętla dla kluczy w sekcji 'district'
    if search_data['props']['pageProps']['ad']['location']['address']['district'] is not None:
        for klucz in klucze_district:
            mieszkanie['district_'+ klucz] = search_data['props']['pageProps']['ad']['location']['address']['district'].get(klucz)
    wszystkie_mieszkanie.append(mieszkanie)
time.sleep(5)


In [69]:
df = pd.DataFrame(wszystkie_mieszkanie)

In [51]:
df.head(5)

,Area,Build_year,Building_floors_num,Building_type,Building_ownership,Construction_status,City,City_id,Country,Extras_types,Floor_no,MarketType,OfferType,Price,Price_per_m,ProperType,Province,Rent,Rooms_num,Subregion,Windows_type,env,Equipment_types,Heating,Media_types,Security_types,Building_material,user_type,id,market,slug,advertType,createdAt,modifiedAt,description,exlusiveOffer,owner_id,owner_name,owner_phones,agency_id,agency_name,latitude,longitude,district_code,district_id,district_name
0,200,1937,1,[house],[full_ownership],[to_renovation],warszawa,26,Polska,[basement],[floor_1],secondary,sprzedaz,2700000.0,13500.0,mieszkanie,mazowieckie,NaN,[8],powiat-warszawa,[wooden],production,None,[gas],None,[entryphone],[brick],agency,33133193,SECONDARY,mieszkanie-200-m-warszawa-ID2f1sl,AGENCY,2014-11-10T12:05:12+01:00,2023-07-25T16:10:55+02:00,<p>SASKA KĘPA - piętro domu - w kameralnym dom...,None,53131,Anna Derda,[+48226163059],53131.0,Magenda Nieruchomości Marek Masny,52.235924,21.047396,saska-kepa,847,Saska Kępa
1,113.8,1987,10,[block],[full_ownership],[ready_to_use],warszawa,26,Polska,"[balcony, basement, garage, lift, separate_kit...",[floor_10],secondary,sprzedaz,1479400.0,13000.0,mieszkanie,mazowieckie,NaN,[4],powiat-warszawa,None,production,[furniture],[urban],"[cable-television, internet]",None,[concrete_plate],agency,41577753,SECONDARY,mieszkanie-113-80-m-warszawa-ID2OsgW,AGENCY,2015-10-27T20:02:18+01:00,2023-08-05T22:00:06+02:00,"<p>*** WYSOKI STANDARD*** 4-pokojowe, wyposażo...",None,570652,Renata Burkacka,[+48667900040],570652.0,Loxton -Real Estate Group,52.141467,21.052604,natolin,743,Natolin
2,120,2022,1,[house],[full_ownership],[to_completion],warszawa,26,Polska,None,[ground_floor],primary,sprzedaz,730000.0,6083.0,mieszkanie,mazowieckie,NaN,[4],powiat-warszawa,[plastic],production,None,[gas],"[internet, phone]","[alarm, anti_burglary_door, closed_area, monit...",[brick],private,43594781,PRIMARY,gotowe-75m2-poddasze-90m2-ul-tawulkowa-7zc-ID2...,PRIVATE,2016-04-07T12:55:57+02:00,2023-08-12T21:23:27+02:00,<p><strong>MIESZKANIA BEZCZYNSZOWE – „OGRODY T...,None,1479135,MH Construction,[+48791660444],NaN,NaN,52.169532,21.182899,wawer,48,Wawer
3,97,2009,6,[apartment],None,None,warszawa,26,Polska,"[balcony, basement, garage, lift, separate_kit...",[floor_2],secondary,sprzedaz,2000000.0,20619.0,mieszkanie,mazowieckie,1800.0,[4],powiat-warszawa,[aluminium],production,None,[urban],None,None,None,agency,47261366,SECONDARY,cztery-pokoje-na-mokotowie-przy-parku-ID3ciQ6,AGENCY,2017-02-20T22:56:32+01:00,2023-08-09T18:50:26+02:00,<p></p><p></p><p>Czteropokojowy apartament w k...,None,2355750,Patrycja Kowacka,[+48668195550],2355750.0,Mansion Nieruchomości,52.202970,21.032850,dolny-mokotow,10150,Dolny Mokotów
4,152,2006,3,[apartment],None,None,warszawa,26,Polska,"[balcony, basement, garage, garden, lift, terr...",[floor_3],secondary,sprzedaz,2799000.0,18414.0,mieszkanie,mazowieckie,NaN,[5],powiat-warszawa,[wooden],production,None,[urban],"[cable-television, internet, phone]",None,None,agency,48075474,SECONDARY,pola-mokotowskie-apartament-z-ogrodem-na-dachu...,AGENCY,2017-04-28T14:18:05+02:00,2023-07-26T09:45:05+02:00,<p><br/>Pięciopokojowy apartament z ogrodem na...,None,99792,Dominik Jeżewski,[+48668498012],99792.0,Kancelaria Nieruchomości EKO HOUSE,52.202180,20.996800,gorny-mokotow,7365,Górny Mokotów


In [28]:
pd.set_option('display.max_columns', None)

Wyposażenie: meble, pralka, zmywarka, lodówka, kuchenka, piekarnik, telewizor
Zabezpieczenia: rolety antywłamaniowe, drzwi / okna antywłamaniowe, domofon / wideofon, monitoring / ochrona, system alarmowy, teren zamknięty
Media: internet, telewizja kablowa, telefon
Informacje dodatkowe: balkon, pom. użytkowe, garaż/miejsce parkingowe, piwnica, ogródek, taras, winda, dwupoziomowe, oddzielna kuchnia, klimatyzacja, tylko dla niepalących


In [70]:
#Tworzę dummies, aby wartości z Extras Types, Media_types i Security_types, podzieliły się na kolumny z wartościami 0,1
dummies_extras = df['Extras_types'].apply(pd.Series).stack().str.get_dummies().groupby(level=0).sum()
dummies_media = df['Media_types'].apply(pd.Series).stack().str.get_dummies().groupby(level=0).sum()
dummies_security = df['Security_types'].apply(pd.Series).stack().str.get_dummies().groupby(level=0).sum()


#Nadpisuje kolumny 'Extras_types', 'Media_types', 'Security_types dummiesami 
df = pd.concat([df.drop(['Extras_types', 'Security_types','Media_types'], axis=1), dummies_extras, dummies_media, dummies_security,], axis=1)


/var/folders/75/8ybrj1sn4rv_ccfth6dy97hw0000gn/T/ipykernel_94737/3494710049.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dummies_extras = df['Extras_types'].apply(pd.Series).stack().str.get_dummies().groupby(level=0).sum()
/var/folders/75/8ybrj1sn4rv_ccfth6dy97hw0000gn/T/ipykernel_94737/3494710049.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dummies_extras = df['Extras_types'].apply(pd.Series).stack().str.get_dummies().groupby(level=0).sum()
/var/folders/75/8ybrj1sn4rv_ccfth6dy97hw0000gn/T/ipykernel_94737/3494710049.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  dummies_extras = df['Extras_types'].apply(pd.Series).stack().

In [76]:
df.head(5)

,Area,Build_year,Building_floors_num,Building_type,Building_ownership,Construction_status,City,City_id,Country,Floor_no,MarketType,OfferType,Price,Price_per_m,ProperType,Province,Rent,Rooms_num,Subregion,Windows_type,env,Equipment_types,Heating,Building_material,user_type,id,market,slug,advertType,createdAt,modifiedAt,description,exlusiveOffer,owner_id,owner_name,owner_phones,agency_id,agency_name,latitude,longitude,district_code,district_id,district_name,air_conditioning,balcony,basement,garage,garden,lift,separate_kitchen,terrace,two_storey,usable_room,cable-television,internet,phone,alarm,anti_burglary_door,closed_area,entryphone,monitoring,roller_shutters
0,200,1937,1,[house],[full_ownership],[to_renovation],warszawa,26,Polska,[floor_1],secondary,sprzedaz,2700000.0,13500.0,mieszkanie,mazowieckie,NaN,[8],powiat-warszawa,[wooden],production,None,[gas],[brick],agency,33133193,SECONDARY,mieszkanie-200-m-warszawa-ID2f1sl,AGENCY,2014-11-10T12:05:12+01:00,2023-07-25T16:10:55+02:00,<p>SASKA KĘPA - piętro domu - w kameralnym dom...,None,53131,Anna Derda,[+48226163059],53131.0,Magenda Nieruchomości Marek Masny,52.235924,21.047396,saska-kepa,847,Saska Kępa,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0
1,113.8,1987,10,[block],[full_ownership],[ready_to_use],warszawa,26,Polska,[floor_10],secondary,sprzedaz,1479400.0,13000.0,mieszkanie,mazowieckie,NaN,[4],powiat-warszawa,None,production,[furniture],[urban],[concrete_plate],agency,41577753,SECONDARY,mieszkanie-113-80-m-warszawa-ID2OsgW,AGENCY,2015-10-27T20:02:18+01:00,2023-08-05T22:00:06+02:00,"<p>*** WYSOKI STANDARD*** 4-pokojowe, wyposażo...",None,570652,Renata Burkacka,[+48667900040],570652.0,Loxton -Real Estate Group,52.141467,21.052604,natolin,743,Natolin,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,120,2022,1,[house],[full_ownership],[to_completion],warszawa,26,Polska,[ground_floor],primary,sprzedaz,730000.0,6083.0,mieszkanie,mazowieckie,NaN,[4],powiat-warszawa,[plastic],production,None,[gas],[brick],private,43594781,PRIMARY,gotowe-75m2-poddasze-90m2-ul-tawulkowa-7zc-ID2...,PRIVATE,2016-04-07T12:55:57+02:00,2023-08-12T21:23:27+02:00,<p><strong>MIESZKANIA BEZCZYNSZOWE – „OGRODY T...,None,1479135,MH Construction,[+48791660444],NaN,NaN,52.169532,21.182899,wawer,48,Wawer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
3,97,2009,6,[apartment],None,None,warszawa,26,Polska,[floor_2],secondary,sprzedaz,2000000.0,20619.0,mieszkanie,mazowieckie,1800.0,[4],powiat-warszawa,[aluminium],production,None,[urban],None,agency,47261366,SECONDARY,cztery-pokoje-na-mokotowie-przy-parku-ID3ciQ6,AGENCY,2017-02-20T22:56:32+01:00,2023-08-09T18:50:26+02:00,<p></p><p></p><p>Czteropokojowy apartament w k...,None,2355750,Patrycja Kowacka,[+48668195550],2355750.0,Mansion Nieruchomości,52.202970,21.032850,dolny-mokotow,10150,Dolny Mokotów,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,152,2006,3,[apartment],None,None,warszawa,26,Polska,[floor_3],secondary,sprzedaz,2799000.0,18414.0,mieszkanie,mazowieckie,NaN,[5],powiat-warszawa,[wooden],production,None,[urban],None,agency,48075474,SECONDARY,pola-mokotowskie-apartament-z-ogrodem-na-dachu...,AGENCY,2017-04-28T14:18:05+02:00,2023-07-26T09:45:05+02:00,<p><br/>Pięciopokojowy apartament z ogrodem na...,None,99792,Dominik Jeżewski,[+48668498012],99792.0,Kancelaria Nieruchomości EKO HOUSE,52.202180,20.996800,gorny-mokotow,7365,Górny Mokotów,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
df['Build_year'].unique()

array(['1937', '1987', '2022', '2009', '2006', None, '2000', '1828',
       '1998', '2005', '1997', '2019', '2018', '2001', '2004', '2012',
       '1995', '2007', '1924', '1928', '1960', '1936', '1912', '1994',
       '2002', '1958', '2020', '1929', '1890', '2023', '2008', '2024',
       '2013', '1992', '1972', '2003', '1970', '2010', '2015', '1953',
       '1957', '1954', '1980', '1999', '2011', '1901', '2021', '1930',
       '2016', '1938', '1910', '1985', '1935', '1974', '1993', '1989',
       '1931', '2017', '1971', '1952', '1939', '1934', '1900', '1990',
       '1', '1965'], dtype=object)